In [5]:
import polars as pl

In [10]:
survey = pl.read_csv('data/health_survey.csv')
reverse = pl.read_csv('data/ReverseCodingItems.csv')

In [15]:
(survey

).head()

ID,F1,F5,F2,F1.1,F2.1,F6,F4,F3,F5.1,F1.2,F2.2,F6.1,F2.3,F4.1,F2.4,F5.2,F2.5,F6.2,F1.3,F2.6,F5.3,F4.2,F2.7,F3.1,F2.8,F5.4,F3.2,F1.4,F3.3,F1.5,F5.5,F6.3,F1.6,F5.6,F2.9,F3.4,F4.3,F2.10,F1.7,F6.4,F4.4,F5.7,F3.5,F2.11
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree"""
2,"""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree"""
3,"""Strongly Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""",null,"""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Disagree""","""Somewhat Agree"""
4,"""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Strongly Agree""","""Neither Agree nor Disagree""","""Neither Agree nor Disagree""","""Somewhat Disagree""","""Somewhat Agree""","""Strongly Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Neither Agree nor Disagree""","""Strongly Agree""","""Somewhat Disagree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Strongly Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Agree""","""Somewhat Disagree""","""Somewh

In [47]:
q_cols = survey.drop('ID').columns

#Question does not need any reverse coding
(reg := {'Strongly Agree':5,
        'Somewhat Agree':4,
        'Neither Agree nor Disagree':3,
        'Somewhat Disagree':2,
        'Strongly Disagree':1,
       }
)

#Question needs reverse coding
(rev := {'Strongly Agree':1,
        'Somewhat Agree':2,
        'Neither Agree nor Disagree':3,
        'Somewhat Disagree':4,
        'Strongly Disagree':5,
       }
)

# QUESTION ONE: STACK DATA
final_df=(
    survey
    .unpivot(
        on=q_cols,
        index="ID",
        variable_name="Question",
        value_name="Response"
    )
    .with_columns(
        reg_coding = pl.col('Response')
            .replace(reg)
            .cast(pl.Int64()),
        rev_coding = pl.col('Response')
            .replace(rev)
            .cast(pl.Int64()),
    )
    .join(
        reverse,
        left_on='Question',
        right_on='Column Name',
        how='left'
    )
    .with_columns(
        coding = pl.when(pl.col('Needs Reverse Coding?') == 'Yes')
            .then('rev_coding')
            .otherwise('reg_coding'),
        q_type = pl.col('Question').str.split('.').list.get(0),
    )
        .pivot(on = 'q_type',
              index='ID',
              values='coding',
              aggregate_function= 'sum',
             )
    
)



In [48]:
final_df.write_csv("data/final_scored_survey.csv")